In [ ]:
import torch 
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# parallel processing
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=16)
from tqdm import tqdm
tqdm.pandas()

# utility
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

In [2]:
from transformers import set_seed
set_seed(42)

In [ ]:
# loading the metrics
import evaluate
exact_match_metric = evaluate.load("exact_match")
sacrebleu = evaluate.load("sacrebleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")


def bleu_compute(pred, suffix):
    return sacrebleu.compute(predictions = pred, references = suffix)["score"]

def em_compute(pred, suffix):
    return exact_match_metric.compute(references=pred, predictions=suffix)['exact_match']

def meteor_compute(pred, suffix):
    return meteor.compute(predictions = pred, references = suffix)["meteor"]

def rouge_compute(pred, suffix):
    return rouge.compute(predictions = pred, references = suffix)["rougeL"]

To change depending on which model you want to attack!

In [4]:
checkpoint = "bigcode/starcoder2-3b"
# checkpoint = "bigcode/starcoder2-7b"
# checkpoint = "bigcode/starcoder2-15b"

# Load the sample datasets for the mem attacks 

In [ ]:
df = pd.read_parquet('mem-tune-replication_package/mem-tune/data/samples/pre-train/forget-attack/pre-train_attack_g3.parquet')
df = Dataset.from_pandas(df[['prefix_250', 'prefix_200', 'prefix_150', 'prefix_100', 'suffix']])

Parameters for this experiment:

- epochs 3
- deduplication rate >3
- we have 4 setups of prefix-length 100, 150, 200, 250


Setup the column names:

- the generation is 7b_ep0_p100, 7b_ep0_p150, 7b_ep3_p250

- for the metric we are just adding the _metric-name to 7b_ep3_p100_bleu, 7b_p150_ep3_meteor...

In [6]:
# setup the experiment names

m_name = checkpoint.split('-')[1] # take only the nymber of parameters

epochs = ['_ep0', '_ep1', '_ep2', '_ep3']

plen = ['_p100', '_p150','_p200','_p250']


# Load the models

In [7]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

**IMPORTANT**

In the public replication package we will also share the fine-tuned models on Huggingface

In [8]:
# Model dirs
#To change depending on which model you want to attack!

ep_dir = '/model_dir/scoder3b/epochs'
ep1 = ep_dir +'/checkpoint-833'
ep2 = ep_dir + '/checkpoint-1666'
ep3= ep_dir + '/checkpoint-2499'

# ep_dir = '/model_dir/scoder7b/epochs'
# ep1 = ep_dir +'/checkpoint-833'
# ep2 = ep_dir + '/checkpoint-1666'
# ep3= ep_dir + '/checkpoint-2499'

# ep_dir = '/model_dir/scoder15b/epochs'
# ep1 = ep_dir +'/checkpoint-800'
# ep2 = ep_dir + '/checkpoint-1600'
# ep3= ep_dir + '/checkpoint-2400'

In [ ]:
# load the model
#To change depending on which model you want to attack!

# base model
model = AutoModelForCausalLM.from_pretrained(checkpoint,device_map="auto")
epoch = epochs[0]

# model depending on the epoch
# model = AutoModelForCausalLM.from_pretrained(ep1,device_map="auto")
# epoch = epochs[1]

# model = AutoModelForCausalLM.from_pretrained(ep2,device_map="auto")
# epoch = epochs[2]

# model = AutoModelForCausalLM.from_pretrained(ep3,device_map="auto")
# epoch = epochs[3]

# eval mode
model.eval()

In [10]:
# define the pipeline
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, framework='pt', max_new_tokens=50)

In [ ]:
# IMPORTANT

gen_name_100 = m_name + epoch + plen[0]
gen_name_150 = m_name + epoch + plen[1]
gen_name_200 = m_name + epoch + plen[2]
gen_name_250 = m_name + epoch + plen[3]

print(f"{gen_name_100}\n {gen_name_150}\n {gen_name_200}\n {gen_name_250}")

# Now we can generate

In [ ]:
gs_mem_100 = pipe(df['prefix_100'], batch_size=32)
print('100 done')
gs_mem_150 = pipe(df['prefix_150'], batch_size=32)
print('150 done')
gs_mem_200 = pipe(df['prefix_200'], batch_size=32)
print('200 done')
gs_mem_250 = pipe(df['prefix_250'], batch_size=32)
print('250 done')

# Save the results on each of the datasets
df = df.add_column(gen_name_100, [gs_mem_100[i][0]['generated_text'][len(df['prefix_100'][i]):]  for i,_ in enumerate(df)])
df = df.add_column(gen_name_150, [gs_mem_150[i][0]['generated_text'][len(df['prefix_150'][i]):]  for i,_ in enumerate(df)])
df = df.add_column(gen_name_200, [gs_mem_200[i][0]['generated_text'][len(df['prefix_200'][i]):]  for i,_ in enumerate(df)])
df = df.add_column(gen_name_250, [gs_mem_250[i][0]['generated_text'][len(df['prefix_250'][i]):]  for i,_ in enumerate(df)])

# Evaluation

In [14]:
expath = 'mem-tune-replication_package/mem-tune/evaluation/forgeting/experiments/attack_forg_plen/attack_forg_plen.parquet'

In [15]:
# Initial attack
df_eval= df.to_pandas()

# After is saved the first time
df_eval = pd.merge((pd.read_parquet(expath + '/attack_forg_plen.parquet')), df.to_pandas()[[gen_name_100, gen_name_150, gen_name_200, gen_name_250]], left_index =True, right_index=True)

plen = 100

In [ ]:
em = gen_name_100 + '_em'
bleu = gen_name_100 + '_bleu'
met = gen_name_100 + '_meteor'
roug = gen_name_100 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval[em] = df_eval.progress_apply(lambda x: em_compute( pred= [x[gen_name_100]], suffix = [x['suffix']]), axis=1)
df_eval[bleu] = df_eval.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_100]], suffix = [x['suffix']]), axis=1)
df_eval[met] = df_eval.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_100]], suffix = [x['suffix']]), axis=1)
df_eval[roug] = df_eval.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_100]], suffix = [x['suffix']]), axis=1)

plen = 150

In [ ]:
em = gen_name_150 + '_em'
bleu = gen_name_150 + '_bleu'
met = gen_name_150 + '_meteor'
roug = gen_name_150 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval[em] = df_eval.progress_apply(lambda x: em_compute( pred= [x[gen_name_150]], suffix = [x['suffix']]), axis=1)
df_eval[bleu] = df_eval.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_150]], suffix = [x['suffix']]), axis=1)
df_eval[met] = df_eval.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_150]], suffix = [x['suffix']]), axis=1)
df_eval[roug] = df_eval.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_150]], suffix = [x['suffix']]), axis=1)

plen = 200

In [ ]:
em = gen_name_200 + '_em'
bleu = gen_name_200 + '_bleu'
met = gen_name_200 + '_meteor'
roug = gen_name_200 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval[em] = df_eval.progress_apply(lambda x: em_compute( pred= [x[gen_name_200]], suffix = [x['suffix']]), axis=1)
df_eval[bleu] = df_eval.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_200]], suffix = [x['suffix']]), axis=1)
df_eval[met] = df_eval.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_200]], suffix = [x['suffix']]), axis=1)
df_eval[roug] = df_eval.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_200]], suffix = [x['suffix']]), axis=1)

plen = 250

In [ ]:
em = gen_name_250 + '_em'
bleu = gen_name_250 + '_bleu'
met = gen_name_250 + '_meteor'
roug = gen_name_250 + '_rougeL'

print(f"{em}\n {bleu}\n {met}\n {roug}")

In [ ]:
# evaluation
df_eval[em] = df_eval.progress_apply(lambda x: em_compute( pred= [x[gen_name_250]], suffix = [x['suffix']]), axis=1)
df_eval[bleu] = df_eval.progress_apply(lambda x: bleu_compute( pred= [x[gen_name_250]], suffix = [x['suffix']]), axis=1)
df_eval[met] = df_eval.progress_apply(lambda x: meteor_compute( pred= [x[gen_name_250]], suffix = [x['suffix']]), axis=1)
df_eval[roug] = df_eval.progress_apply(lambda x: rouge_compute( pred= [x[gen_name_250]], suffix = [x['suffix']]), axis=1)

In [25]:
# Save the results
df_eval.to_parquet(os.path.join(expath, 'attack_forg_plen.parquet'))